In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cpi
cpi.update()
from datetime import date
from sqlalchemy import create_engine
import tensorflow as tf

In [ ]:
# Read in data
tornado_data = pd.read_csv('Tornadoes_SPC_1950to2015.csv')
tornado_data.columns

In [ ]:
# Clean up file
tornado_data.rename(columns = {'yr':'Year', 'mo':'Month','dy':'Day','date':'Date','time':'Time','tz':'Time_Zone',
                               'st':'State','stf':'State_FIPS','stn':'State_No','mag':'Magnitude','inj':'Injuries',
                               'fat':'Fatalities','loss':'Property_Loss','closs':'Crop_Loss','slat':'Starting_Lat',
                               'slon':'Starting_Lon','elat':'Ending_Lat','elon':'Ending_Lon','len':'Length',
                               'wid':'Width'}, inplace = True)
tornado_data

In [4]:
# Remove zeroes from Property Loss column
tornado_data = tornado_data[tornado_data['Property_Loss'] > 0]
tornado_data

,om,Year,Month,Day,Date,Time,Time_Zone,State,State_FIPS,State_No,...,Fatalities,Property_Loss,Crop_Loss,Starting_Lat,Starting_Lon,Ending_Lat,Ending_Lon,Length,Width,fc
0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,1,...,0,6.00,0.0,38.77,-90.22,38.83,-90.03,9.50,150,0
1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,...,0,5.00,0.0,39.10,-89.30,39.12,-89.23,3.60,130,0
2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,1,...,0,4.00,0.0,40.88,-84.58,0.00,0.00,0.10,10,0
3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,...,1,3.00,0.0,34.40,-94.37,0.00,0.00,0.60,17,0
4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,...,0,5.00,0.0,37.60,-90.68,37.63,-90.65,2.30,300,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60106,607040,2015,12,27,12/27/2015,20:16:00,3,LA,22,0,...,0,0.25,0.0,32.61,-93.88,32.64,-93.88,2.27,144,0
60110,613277,2015,12,28,12/28/2015,4:46:00,3,AR,5,0,...,0,0.05,0.0,34.70,-90.92,34.78,-90.90,5.75,40,0
60111,607307,2015,12,28,12/28/2015,5:43:00,3,MS,28,0,...,0,0.10,0.0,31.54,-89.53,31.61,-89.49,5.59,100,0
60112,612738,2015,12,28,12/28/2015,8:30:00,3,FL,12,0,...,0,0.01,0.0,30.76,-87.24,30.77,-87.23,0.78,75,0


In [5]:
# Export data to csv to process through python and add Census API data
tornado_data.to_csv('tornado_data_no_zeroes.csv')

In [6]:
# Read in csv with Census data added
tornado_data = pd.read_csv('tornado_data_census.csv')
tornado_data

,om,Year,Month,Day,Date,Time,Time_Zone,State,State_FIPS,State_No,...,Crop_Loss,Starting_Lat,Starting_Lon,Ending_Lat,Ending_Lon,Length,Width,fc,Income,Pop_Density
0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,1,...,0.0,38.77,-90.22,38.83,-90.03,9.50,150,0,35368.0,3115.77
1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,...,0.0,39.10,-89.30,39.12,-89.23,3.60,130,0,49840.0,24.06
2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,1,...,0.0,40.88,-84.58,0.00,0.00,0.10,10,0,36734.0,767.15
3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,...,0.0,34.40,-94.37,0.00,0.00,0.60,17,0,37621.0,10.65
4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,...,0.0,37.60,-90.68,37.63,-90.65,2.30,300,0,37614.0,27.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36314,607040,2015,12,27,12/27/2015,20:16:00,3,LA,22,0,...,0.0,32.61,-93.88,32.64,-93.88,2.27,144,0,65188.0,515.20
36315,613277,2015,12,28,12/28/2015,4:46:00,3,AR,5,0,...,0.0,34.70,-90.92,34.78,-90.90,5.75,40,0,33167.0,8.00
36316,607307,2015,12,28,12/28/2015,5:43:00,3,MS,28,0,...,0.0,31.54,-89.53,31.61,-89.49,5.59,100,0,34676.0,40.85
36317,612738,2015,12,28,12/28/2015,8:30:00,3,FL,12,0,...,0.0,30.76,-87.24,30.77,-87.23,0.78,75,0,78587.0,175.61


In [ ]:
# Remove any NANs from Census columns
tornado_data = tornado_data.replace(r'^s*$', float('NaN'), regex = True)
tornado_data.dropna(inplace = True)
tornado_data

In [7]:
# Translate Property Loss to $ amounts - use midpoint in each range, translate to fraction of million
# 1 = .000025
# 2 = .000275
# 3 = .00275
# 4 = .0275
# 5 = .275
# 6 = 2.75
# 7 = 27.5
# 8 = 275
# 9 = 2750

tornado_data['Property_Loss'] = tornado_data['Property_Loss'].replace([1, 2, 3, 4, 5, 6, 7, 8, 9], 
                                    [.000025, .000275, .00275, .0275, .275, 2.75, 27.5, 275, 2750])
tornado_data


,om,Year,Month,Day,Date,Time,Time_Zone,State,State_FIPS,State_No,...,Crop_Loss,Starting_Lat,Starting_Lon,Ending_Lat,Ending_Lon,Length,Width,fc,Income,Pop_Density
0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,1,...,0.0,38.77,-90.22,38.83,-90.03,9.50,150,0,35368.0,3115.77
1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,...,0.0,39.10,-89.30,39.12,-89.23,3.60,130,0,49840.0,24.06
2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,1,...,0.0,40.88,-84.58,0.00,0.00,0.10,10,0,36734.0,767.15
3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,...,0.0,34.40,-94.37,0.00,0.00,0.60,17,0,37621.0,10.65
4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,...,0.0,37.60,-90.68,37.63,-90.65,2.30,300,0,37614.0,27.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36314,607040,2015,12,27,12/27/2015,20:16:00,3,LA,22,0,...,0.0,32.61,-93.88,32.64,-93.88,2.27,144,0,65188.0,515.20
36315,613277,2015,12,28,12/28/2015,4:46:00,3,AR,5,0,...,0.0,34.70,-90.92,34.78,-90.90,5.75,40,0,33167.0,8.00
36316,607307,2015,12,28,12/28/2015,5:43:00,3,MS,28,0,...,0.0,31.54,-89.53,31.61,-89.49,5.59,100,0,34676.0,40.85
36317,612738,2015,12,28,12/28/2015,8:30:00,3,FL,12,0,...,0.0,30.76,-87.24,30.77,-87.23,0.78,75,0,78587.0,175.61


In [8]:
# Adjust $ amount for inflation
tornado_data["Adj_Property_Loss"] = tornado_data.apply(lambda x: cpi.inflate(x["Property_Loss"], x["Year"]), axis=1)

tornado_data

,om,Year,Month,Day,Date,Time,Time_Zone,State,State_FIPS,State_No,...,Starting_Lat,Starting_Lon,Ending_Lat,Ending_Lon,Length,Width,fc,Income,Pop_Density,Adj_Property_Loss
0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,1,...,38.77,-90.22,38.83,-90.03,9.50,150,0,35368.0,3115.77,29.532376
1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,...,39.10,-89.30,39.12,-89.23,3.60,130,0,49840.0,24.06,2.953238
2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,1,...,40.88,-84.58,0.00,0.00,0.10,10,0,36734.0,767.15,0.295324
3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,...,34.40,-94.37,0.00,0.00,0.60,17,0,37621.0,10.65,0.029532
4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,...,37.60,-90.68,37.63,-90.65,2.30,300,0,37614.0,27.70,2.953238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36314,607040,2015,12,27,12/27/2015,20:16:00,3,LA,22,0,...,32.61,-93.88,32.64,-93.88,2.27,144,0,65188.0,515.20,0.272988
36315,613277,2015,12,28,12/28/2015,4:46:00,3,AR,5,0,...,34.70,-90.92,34.78,-90.90,5.75,40,0,33167.0,8.00,0.054598
36316,607307,2015,12,28,12/28/2015,5:43:00,3,MS,28,0,...,31.54,-89.53,31.61,-89.49,5.59,100,0,34676.0,40.85,0.109195
36317,612738,2015,12,28,12/28/2015,8:30:00,3,FL,12,0,...,30.76,-87.24,30.77,-87.23,0.78,75,0,78587.0,175.61,0.010920


In [16]:
import requests
import json
from config import api_key
def get_json_zipcode2(df, lat_field, lon_field):
    try:
        url = f'https://api.bigdatacloud.net/data/reverse-geocode?latitude={df[lat_field]}&longitude={df[lon_field]}&localityLanguage=en&key={api_key}'
#         print(url)
        location = requests.get(url).json()
#         print(location['postcode'])
        return location['postcode']
#     except GeocoderTimedOut:
#         if attempt <= max_attempts:
#             time.sleep(5)
#             print(f'attempt number {attempt}')
#             return get_zipcode(df, lat_field, lon_field, attempt=attempt+1)
#         raise
    except:
        return 0
#     return location.raw['address']['postcode']

In [17]:
tornado_data['zipcode'] = tornado_data.apply(get_json_zipcode2, axis=1, lat_field='Starting_Lat', lon_field='Starting_Lon')
tornado_data

,om,Year,Month,Day,Date,Time,Time_Zone,State,State_FIPS,State_No,...,Starting_Lon,Ending_Lat,Ending_Lon,Length,Width,fc,Income,Pop_Density,Adj_Property_Loss,zipcode
0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,1,...,-90.22,38.83,-90.03,9.50,150,0,35368.0,3115.77,29.532376,63138
1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,...,-89.30,39.12,-89.23,3.60,130,0,49840.0,24.06,2.953238,62032
2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,1,...,-84.58,0.00,0.00,0.10,10,0,36734.0,767.15,0.295324,45891
3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,...,-94.37,0.00,0.00,0.60,17,0,37621.0,10.65,0.029532,71937
4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,...,-90.68,37.63,-90.65,2.30,300,0,37614.0,27.70,2.953238,63650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36314,607040,2015,12,27,12/27/2015,20:16:00,3,LA,22,0,...,-93.88,32.64,-93.88,2.27,144,0,65188.0,515.20,0.272988,71107
36315,613277,2015,12,28,12/28/2015,4:46:00,3,AR,5,0,...,-90.92,34.78,-90.90,5.75,40,0,33167.0,8.00,0.054598,72360
36316,607307,2015,12,28,12/28/2015,5:43:00,3,MS,28,0,...,-89.53,31.61,-89.49,5.59,100,0,34676.0,40.85,0.109195,39479
36317,612738,2015,12,28,12/28/2015,8:30:00,3,FL,12,0,...,-87.24,30.77,-87.23,0.78,75,0,78587.0,175.61,0.010920,32571


In [22]:
df_test = tornado_data.loc[tornado_data['zipcode'] == 0]
df_test

,om,Year,Month,Day,Date,Time,Time_Zone,State,State_FIPS,State_No,...,Starting_Lon,Ending_Lat,Ending_Lon,Length,Width,fc,Income,Pop_Density,Adj_Property_Loss,zipcode


In [23]:
print(len(tornado_data))
tornado_data = tornado_data.dropna(how='any')
print(len(tornado_data))

36319
36182


In [25]:
tornado_data = tornado_data.loc[tornado_data['zipcode'] != 0]
print(len(tornado_data))

36182


In [26]:
tornado_data.to_csv('tornado_data_zipcode.csv')

In [ ]:
##############  Start of Database creation  #############

In [9]:
tornado_data = pd.read_csv('tornado_data_zipcode.csv')
tornado_data=tornado_data.rename(columns={"Unnamed: 0": "index"})
tornado_data

,index,om,Year,Month,Day,Date,Time,Time_Zone,State,State_FIPS,...,Starting_Lon,Ending_Lat,Ending_Lon,Length,Width,fc,Income,Pop_Density,Adj_Property_Loss,zipcode
0,0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,...,-90.22,38.83,-90.03,9.50,150,0,35368.0,3115.77,29.532376,63138.0
1,1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,...,-89.30,39.12,-89.23,3.60,130,0,49840.0,24.06,2.953238,62032.0
2,2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,...,-84.58,0.00,0.00,0.10,10,0,36734.0,767.15,0.295324,45891.0
3,3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,...,-94.37,0.00,0.00,0.60,17,0,37621.0,10.65,0.029532,71937.0
4,4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,...,-90.68,37.63,-90.65,2.30,300,0,37614.0,27.70,2.953238,63650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36177,36314,607040,2015,12,27,12/27/2015,20:16:00,3,LA,22,...,-93.88,32.64,-93.88,2.27,144,0,65188.0,515.20,0.272988,71107.0
36178,36315,613277,2015,12,28,12/28/2015,4:46:00,3,AR,5,...,-90.92,34.78,-90.90,5.75,40,0,33167.0,8.00,0.054598,72360.0
36179,36316,607307,2015,12,28,12/28/2015,5:43:00,3,MS,28,...,-89.53,31.61,-89.49,5.59,100,0,34676.0,40.85,0.109195,39479.0
36180,36317,612738,2015,12,28,12/28/2015,8:30:00,3,FL,12,...,-87.24,30.77,-87.23,0.78,75,0,78587.0,175.61,0.010920,32571.0


In [10]:
# create database
from config import password
connection_string = f"postgres:{password}@localhost:5432/tornado"
# connection_string = f"postgres:postgres@localhost:5432/Tornado"
engine = create_engine(f'postgresql://{connection_string}')

In [11]:
# Put data into database
tornado_data.to_sql(name='tornado_data', con=engine, if_exists='replace', index=False)

In [12]:
pd.read_sql_query('select * from tornado_data', con=engine).head()

,index,om,Year,Month,Day,Date,Time,Time_Zone,State,State_FIPS,...,Starting_Lon,Ending_Lat,Ending_Lon,Length,Width,fc,Income,Pop_Density,Adj_Property_Loss,zipcode
0,0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,...,-90.22,38.83,-90.03,9.5,150,0,35368.0,3115.77,29.532376,63138.0
1,1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,...,-89.30,39.12,-89.23,3.6,130,0,49840.0,24.06,2.953238,62032.0
2,2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,...,-84.58,0.00,0.00,0.1,10,0,36734.0,767.15,0.295324,45891.0
3,3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,...,-94.37,0.00,0.00,0.6,17,0,37621.0,10.65,0.029532,71937.0
4,4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,...,-90.68,37.63,-90.65,2.3,300,0,37614.0,27.70,2.953238,63650.0
